In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import seaborn as sns
import ee
import pyproj

In [ ]:
population_df = pd.read_csv('population.csv', index_col=0)

In [8]:
ee.Initialize(project='ee-jonesd6000')

In [ ]:
# =============================================================================
# Code generated by Artificial Intelligence (Google Gemini)
# Date: 2025-12-02
# Prompt: "Python function to download elevation data from earthengine-api."
# Disclaimer: This code was reviewed and verified by a human developer. 
# =============================================================================

import pandas as pd
import ee
# ee.Initialize() should be called once before this loop

GLO30 = "COPERNICUS/DEM/GLO30"
# The GLO30 asset is an ImageCollection that needs to be mosaicked into a single image
dem_image = ee.ImageCollection(GLO30).mosaic()
# The band name for elevation in GLO30 is typically 'elevation' or is implicitly the first band.

dem_data_list = []

# Iterate over a sample of points from your population DataFrame
for i, row in population_df.iterrows():
    point = ee.Geometry.Point(row['LONG'], row['LAT'])
    
    # Use reduceRegion directly on the static mosaicked image
    # We specify a scale because the default (1 degree) is too coarse for GLO30's 30m resolution
    elevation_value_dict = dem_image.reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=point,
        scale=30 # Sample at the native 30m resolution
    ).getInfo()

    # The GLO30 image has one band, which defaults to a name like 'elevation' or the asset ID prefix if unspecified.
    # We retrieve the single value from the returned dictionary
    elevation = elevation_value_dict[list(elevation_value_dict.keys())[0]]
    
    # Store the results in a dictionary and append to a list
    dem_data_list.append({
        'LONG': row['LONG'],
        'LAT': row['LAT'],
        'Elevation (m)': elevation
    })

# Convert the list of results into a final Pandas DataFrame
dem_df = pd.DataFrame(dem_data_list)

print(dem_df.head())


In [ ]:
fig = plt.figure(figsize=(20,9))
_map = plt.axes(projection=ccrs.PlateCarree())
_map.set_global()
_map.stock_img() # Adds a low-res natural earth shaded relief background
_map.coastlines()
plt.show()
